In [ ]:
!git clone https://github.com/jan1na/Neural-Cellular-Automata.git

%cd Neural-Cellular-Automata

Cloning into 'Neural-Cellular-Automata'...
remote: Enumerating objects: 102, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 102 (delta 61), reused 43 (delta 23), pack-reused 0 (from 0)
Receiving objects: 100% (102/102), 333.17 KiB | 1.34 MiB/s, done.
Resolving deltas: 100% (61/61), done.
/content/Neural-Cellular-Automata


In [ ]:
!pip install -U gdown

file_ids = ['1W7y-7ebqUCFW2ZPEsafABw_OPnXRr6cj',
            '1L99QItxhTDj3dS1dd8ubmxka4FxXb2HG',
            '1PqrOpn4NJF3xp4Jw2KtRRVD_20Feucsc',
            '1DtZJ5eD2PV4sO51MMQy9CVTZxgBKj_Gx']

file_names = ['pathmnist.npz', 'pathmnist_64.npz', 'pathmnist_128.npz', 'pathmnist_224.npz']

for file_id, file_name in zip(file_ids, file_names):
    !gdown --id {file_id} --output {file_name}

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1W7y-7ebqUCFW2ZPEsafABw_OPnXRr6cj
From (redirected): https://drive.google.com/uc?id=1W7y-7ebqUCFW2ZPEsafABw_OPnXRr6cj&confirm=t&uuid=9d64e7b1-eb18-4a93-aea4-2575b7964729
To: /content/Neural-Cellular-Automata/pathmnist.npz
100% 206M/206M [00:03<00:00, 57.0MB/s]
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1L99QItxhTDj3dS1dd8ubmxka4FxXb2HG
From (redirected): https://drive.google.com/uc?id=1L99QItxhTDj3dS1dd8ubmxka4FxXb2HG&confirm=t&uuid=498ec29a-d3fe-42fc-9691-37a59631f2ef
T

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q medmnist scikit-learn

import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchmetrics.classification import MulticlassCalibrationError
from medmnist import PathMNIST
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, balanced_accuracy_score, mean_absolute_error, brier_score_loss
from sklearn.preprocessing import label_binarize
import seaborn as sns
from models import NCA, CNNBaseline
import torch.nn.functional as F
import numpy as np



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
from google.colab import drive
import os
import shutil

drive_folder = "/content/Neural-Cellular-Automata/"
cache_dir = os.path.expanduser("~/.medmnist")
os.makedirs(cache_dir, exist_ok=True)

resolutions = ["", "_64", "_128", "_224"]
for res in resolutions:
    filename = f"pathmnist{res}.npz"
    src = os.path.join(drive_folder, filename)
    dst = os.path.join(cache_dir, filename)

    if os.path.exists(src):
        shutil.copyfile(src, dst)
        print(f"Copied {filename} to cache.")
    else:
        print(f"File not found in Drive: {filename}")

Copied pathmnist.npz to cache.
Copied pathmnist_64.npz to cache.
Copied pathmnist_128.npz to cache.
Copied pathmnist_224.npz to cache.


In [ ]:
nca = NCA().to(device)
nca.load_state_dict(torch.load("/content/drive/MyDrive/NCA/best_nca_pathmnist.pth"))
nca.eval()

cnn = CNNBaseline().to(device)
cnn.load_state_dict(torch.load("/content/drive/MyDrive/NCA/best_cnn_pathmnist.pth"))
cnn.eval()

CNNBaseline(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
  )
  (pool): AdaptiveAvgPool2d(output_size=(1, 1))
  (fc): Linear(in_features=128, out_features=9, bias=True)
)

In [ ]:
def get_loader(size, batch_size=64):
    transform = transforms.Compose([transforms.ToTensor()])
    dataset = PathMNIST(split="test", size=size, download=False, transform=transform)
    return DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [ ]:
@torch.no_grad()
def evaluate(model, loader, name="Model", size=28, save_dir="/content/drive/MyDrive/NCA/results", is_NCA=False):
    all_preds, all_labels, all_logits = [], [], []

    for x, y in loader:
        x, y = x.to(device), y.squeeze()
        if is_NCA:
            out, rgb_steps = model(x, True)
        else:
            out = model(x)

        all_logits.append(out.cpu())
        pred = out.argmax(dim=1).cpu().numpy()
        label = y.numpy()
        all_preds.extend(pred)
        all_labels.extend(label)

    # Converting for scores
    logits = torch.cat(all_logits)  # (N, C)
    probs = F.softmax(logits, dim=1).numpy()  # (N, C)
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    # Metrics
    cm = confusion_matrix(all_labels, all_preds)
    overall_acc = accuracy_score(all_labels, all_preds)
    bal_acc = balanced_accuracy_score(all_labels, all_preds)
    mae = mean_absolute_error(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, digits=4)


    ### Uncertainty Quantification (NQM)

    # Brier Score
    y_true_bin = label_binarize(all_labels, classes=list(range(probs.shape[1])))
    brier = np.mean(np.sum((probs - y_true_bin) ** 2, axis=1))

    # NLL (Cross-Entropy)
    all_labels_tensor = torch.tensor(all_labels)
    nll = F.cross_entropy(logits, all_labels_tensor, reduction='mean').item()

    # Entroyp
    entropy = -np.sum(probs * np.log(probs + 1e-12), axis=1)
    mean_entropy = np.mean(entropy)

    # ECE
    probs_tensor = torch.from_numpy(probs).float()
    labels_tensor = torch.from_numpy(all_labels).long()

    ece_metric = MulticlassCalibrationError(num_classes=probs.shape[1], n_bins=15, norm='l1')
    ece = ece_metric(probs_tensor, labels_tensor).item()


    # Logging
    print(f"\n{name} @ {size}x{size}")
    print(f"Overall Accuracy: {overall_acc:.4f}")
    print("Balanced Accuracy:", f"{bal_acc:.4f}")
    print("Mean Absolute Error (MAE):", f"{mae:.4f}")
    print("")
    print("Uncertainty Quantification:")
    print(f"Brier Score: {brier:.4f}")
    print(f"NLL (Cross-Entropy)  : {nll:.4f}")
    print(f"Mean Predictive Entropy: {mean_entropy:.4f}")
    print(f"Expected Calibration Error (ECE): {ece:.4f}")

    print(report)

    # Save confusion matrix
    os.makedirs(save_dir, exist_ok=True)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f"Confusion Matrix: {name} @ {size}x{size}")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    fname = f"{save_dir}/cm_{name.replace(' ', '_')}_{size}x{size}.png"
    plt.savefig(fname)
    plt.close()
    print(f"Confusion matrix saved to: {fname}")

In [ ]:
for size in [28, 64, 128, 224]:
    print(f"\n==============================")
    print(f"Resolution: {size}x{size}")
    loader = get_loader(size)

    print("CNN:")
    evaluate(cnn, loader, name="CNN", size=size)

    print("NCA:")
    evaluate(nca, loader, name="NCA", size=size, is_NCA=True)


Resolution: 28x28
CNN:

CNN @ 28x28
Overall Accuracy: 0.8294
Balanced Accuracy: 0.7564
Brier Score: 0.2654
NLL (Cross-Entropy)  : 0.7352
Mean Absolute Error (MAE): 0.5134
              precision    recall  f1-score   support

           0     0.9658    0.9910    0.9782      1338
           1     0.9207    1.0000    0.9587       847
           2     0.3044    0.4248    0.3547       339
           3     0.8541    0.8770    0.8654       634
           4     0.9390    0.8329    0.8827      1035
           5     0.6284    0.5541    0.5889       592
           6     0.9127    0.7193    0.8045       741
           7     0.5886    0.4656    0.5199       421
           8     0.8272    0.9432    0.8814      1233

    accuracy                         0.8294      7180
   macro avg     0.7712    0.7564    0.7594      7180
weighted avg     0.8363    0.8294    0.8292      7180

Confusion matrix saved to: /content/drive/MyDrive/NCA/results/cm_CNN_28x28.png
NCA:

NCA @ 28x28
Overall Accuracy: 0.8981
B

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



NCA @ 224x224
Overall Accuracy: 0.2302
Balanced Accuracy: 0.2468
Brier Score: 1.2015
NLL (Cross-Entropy)  : 4.4813
Mean Absolute Error (MAE): 2.3319
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      1338
           1     0.2609    0.9079    0.4053       847
           2     0.2256    0.6814    0.3390       339
           3     0.0000    0.0000    0.0000       634
           4     0.0370    0.0058    0.0100      1035
           5     0.1977    0.0287    0.0501       592
           6     0.0000    0.0000    0.0000       741
           7     0.2910    0.1306    0.1803       421
           8     0.2109    0.4663    0.2905      1233

    accuracy                         0.2302      7180
   macro avg     0.1359    0.2468    0.1417      7180
weighted avg     0.1163    0.2302    0.1298      7180

Confusion matrix saved to: /content/drive/MyDrive/NCA/results/cm_NCA_224x224.png
